In [1]:
# Previous code wasn't working well.

In [2]:
import networkx as nx
import pandas as pd
import os

In [3]:
# Change this line as needed.
data_folder = "C:/Users/alder/data/"

In [4]:
# This takes a little bit to run.
string_fp = data_folder + "networks/STRING.edg"
g = nx.read_weighted_edgelist(string_fp)

In [5]:
# The functions to find the three network properties I am interested in.
def find_number_of_genes(g):
    return len(g.nodes)

def find_edge_density(g):
    """
    Returns the edge density of graph g.
    """
    
    # This doesn't give me anything that I want.
    # I'll stick with the old return statement for now.
    # The shape was right and the numbers were close.
    """
    total_weight = 0
    for edge in list(g.edges.data()): 
        total_weight = edge[2]['weight']
    possible_weight = len(g.nodes) * (len(g.nodes)-1) / 2
    return total_weight / possible_weight
    """
    # The old return statement
    return len(g.edges) / (len(g.nodes) * (len(g.nodes)-1))

def find_segregation(g, total_g):
    """
    Args:
        g: the subgraph that represents the labeled genes.
        total_g: the entire subgraph in the geneset
            Note: I am unsure if this is what I need, but it might be.
    """    
    inner_edges = len(g.edges)
    outer_edges = 0
    
    # This counts edges within the network twice, and outside once.
    for node in g.nodes:
        outer_edges += len(total_g.edges(node))
    outer_edges = outer_edges - inner_edges * 2
    # We counted inside edges twice, hence  ^^^
    
    possible_outer_edges = len(g.nodes) * (len(total_g.nodes) - len(g.nodes) )
    return outer_edges / possible_outer_edges


In [6]:
# Gets the names and paths for the six different folders in the labels directory.
names = []
paths = []

labels_fp = data_folder + "labels/"
for folder in os.listdir(labels_fp):
    paths.append(labels_fp + folder + "/STRING")
    names.append(folder)

In [7]:
# Here is what one file looks like.
test_df = pd.read_csv(paths[0] + "/" + os.listdir(paths[0])[0], delimiter = '\t')
test_df

,Entrez Gene ID,Label,Train or Test
0,10005,0,Ts
1,100125288,0,Ts
2,10013,0,Tr
3,100130958,0,Ts
4,10020,0,Tr
...,...,...,...
2394,9966,0,Tr
2395,9968,0,Tr
2396,9971,0,Tr
2397,9992,0,Tr


In [ ]:
#...and here is the code to open every file.
dataframes = []

for (name, path) in zip(names, paths):
    dataframes.append(pd.DataFrame(columns = ["Geneset", "Number of Genes", "Edge Density", "Segregation"]))
    for file in os.listdir(path):
        some_df = pd.read_csv(path + "/" + file, delimiter = '\t')
        new_g = g.subgraph(some_df[some_df["Label"] == 1]["Entrez Gene ID"].astype(str).values)
        new_g_total = g.subgraph(some_df["Entrez Gene ID"].astype(str).values)  # May not need, idk
        new_row = [file[:-4],  # The indexing removes the .tsv
                   find_number_of_genes(new_g),
                   find_edge_density(new_g),
                   find_segregation(new_g, new_g_total)]
        new_row = pd.Series(new_row, index = dataframes[-1].columns)
        
        dataframes[-1] = dataframes[-1].append(new_row, ignore_index = True)

In [ ]:
# Saves all of these into this file's directory.
for (name, dataframe) in zip(names, dataframes):
    dataframe.to_csv(name + '_STRING2.tsv', index=False, sep = "\t")